In [ ]:
from dotenv import load_dotenv
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate\
from langchain_classic import hub
from langchain_classic.chains import RetrievalQA

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

index_name = "tax-makrdown-index"

database = PineconeVectorStore.from(document_list, embedding, index_name=index_name)
llm = ChatOpenAI(model='gpt-4o')

prompt = hub.pull("rlm/rag-prompt")

retriever = database.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt}
)


ai_message = qa_chain({'query':query})

dictionary=["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변결할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

tax_chain = {"query": dictionary_chain} | qa_chain

True